In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rnd


In [ ]:
df = pd.read_csv('new_dataframe.csv', sep = ',')

In [ ]:
df.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101500 entries, 0 to 101499
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         101500 non-null  int64  
 1   user_id            101500 non-null  int64  
 2   hour               101500 non-null  int64  
 3   os                 101500 non-null  object 
 4   order_class        101500 non-null  object 
 5   surge              91431 non-null   object 
 6   app_opened         101500 non-null  int64  
 7   price_seen         101500 non-null  int64  
 8   order_made         101500 non-null  int64  
 9   ride_completed     101500 non-null  int64  
 10  user_cancelled     101500 non-null  int64  
 11  city_center_order  101500 non-null  int64  
 12  distance           91431 non-null   float64
 13  age                101500 non-null  int64  
 14  rfm                101500 non-null  object 
dtypes: float64(1), int64(10), object(4)
memory usage: 1

In [ ]:
df.describe()

,Unnamed: 0,user_id,hour,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age
count,101500.000000,101500.000000,101500.000000,101500.0,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,91431.000000,101500.000000
mean,50749.500000,549874.802207,11.481429,1.0,0.900798,0.731389,0.620365,0.111025,0.571675,5.371152,25.906384
std,29300.670499,258600.104176,6.917522,0.0,0.298934,0.443239,0.485299,0.314164,0.494838,4.118531,7.838328
min,0.000000,100093.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010946,18.000000
25%,25374.750000,328223.000000,5.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,2.438335,20.000000
50%,50749.500000,548827.000000,11.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,4.279565,24.000000
75%,76124.250000,773051.000000,17.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,7.129814,29.000000
max,101499.000000,999978.000000,23.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,40.268966,69.000000


# Особенность №1 В период Surge видим более низкую price-to-order конверсию
Гипотеза: Если в период Surge предоставить пользователю скидку в 7%, то это увеличит конверсию price-to-order на 5% за счет более привлекательной цены

In [ ]:
#создаем датафрейм по заказам, совершенным в период Surge
df_surge = df[df['surge'] == 'surge']
df_surge.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
9,9,662775,17,Android,comfort,surge,1,1,1,1,0,1,4.828326,36,low
10,10,464055,20,iOS,comfort,surge,1,1,1,1,0,0,3.647097,25,high
18,18,393096,23,Android,comfort,surge,1,1,0,0,0,1,2.523357,19,high
21,21,103860,13,iOS,economy,surge,1,1,1,1,0,1,7.934002,21,high
24,24,461185,21,iOS,economy,surge,1,1,1,1,0,1,1.590209,29,high


In [ ]:
#посчитаем среднюю конверсию нажатия кнопки совершения заказа пользователями, которые увидели стоимость поездки в период высокого спроса
df_mean_conv = df_surge.loc[(df_surge['price_seen'] == 1)].groupby(['user_id'])['order_made'].mean().reset_index()
df_mean_conv.head()

,user_id,order_made
0,100093,0.5
1,100118,1.0
2,100134,1.0
3,100166,1.0
4,100255,0.0


In [ ]:
#произведем рандомную разбивку пользователей на 2 группы
df_mean_conv['random_value'] = np.random.uniform(0,1, size = len(df_mean_conv['user_id']))


In [ ]:
def groups_gen(row):
  if row['random_value'] <=0.5:
    return(1)
  else: 
    return(2)

In [ ]:
df_mean_conv['group'] = df_mean_conv.apply(groups_gen, axis=1)

In [ ]:
df_mean_conv.head()

,user_id,order_made,random_value,group
0,100093,0.5,0.991589,2
1,100118,1.0,0.321682,1
2,100134,1.0,0.305184,1
3,100166,1.0,0.075794,1
4,100255,0.0,0.655822,2


In [ ]:
df_mean_conv_g1 = df_mean_conv[df_mean_conv['group'] == 1]
df_mean_conv_g2 = df_mean_conv[df_mean_conv['group'] == 2]

In [ ]:
df_mean_conv_g1.head()

,user_id,order_made,random_value,group
1,100118,1.0,0.321682,1
2,100134,1.0,0.305184,1
3,100166,1.0,0.075794,1
5,100275,0.0,0.275323,1
6,100313,1.0,0.067742,1


In [ ]:
df_mean_conv_g2.head()

,user_id,order_made,random_value,group
0,100093,0.500000,0.991589,2
4,100255,0.000000,0.655822,2
9,100424,1.000000,0.763005,2
10,100434,0.666667,0.800432,2
11,100452,0.500000,0.868193,2


In [ ]:
# проверим размеры выборок и выровняем их
len(df_mean_conv_g1['user_id'])

8903

In [ ]:
len(df_mean_conv_g2['user_id'])

8877

In [ ]:
# первая группа имеет больше пользователей. Сократим их количество, чтобы оно было равно количеству пользователей во 2-й группе
group_volume = min(len(df_mean_conv_g1['user_id']), len(df_mean_conv_g2['user_id']))

In [ ]:
df_mean_conv_g1 = df_mean_conv_g1.iloc[0:group_volume-1]
df_mean_conv_g2 = df_mean_conv_g2.iloc[0:group_volume-1]

In [ ]:
len(df_mean_conv_g1['user_id'])

8876

In [ ]:
len(df_mean_conv_g2['user_id'])

8876

Проверяем группы на однородность с помощью раногового критерия Манна-Уитни

In [ ]:
#Формируем H0 и H1:
#H0 Группы 1 и 2 являются однородными
#H1 Группы 1 и 2 имеют статистически значимые различия

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
alpha = 0.05
stat, p = mannwhitneyu(df_mean_conv_g1['order_made'], df_mean_conv_g2['order_made'])
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > alpha:
  print('Группы 1 и 2 являются однородными (не отвергаем H0)')
else:
  print('Группы 1 и 2 имеют статистически значимые различия (отвергаем H0)')


Statistics=38936439.500, p=0.133
Группы 1 и 2 являются однородными (не отвергаем H0)


# Особенность №2 Если Заказ из центра, то видим более низкие юзерские отмены 
Гипотеза: Если мы увеличим стоимость поездки на 10% при заказах из центра в период с 21-00 до 03-00, наша прибыль увеличится на 5%, т.к. в это время пользователи менее склонны к отменам поездки.

In [ ]:
#создаем датафрейм по заказам из центра, совершенным в промежуток времени с 21-00 до 03-00
df_center_eve = df[(df['city_center_order'] == 1) & ((df['hour'] >= 0) & (df['hour'] <= 3) | (df['hour'] >= 21))]
df_center_eve.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
6,6,416988,0,iOS,comfort,no surge,1,1,0,0,0,1,7.535725,24,high
7,7,682337,2,Android,comfort,no surge,1,1,1,1,0,1,9.055344,21,low
13,13,786923,23,Android,comfort,no surge,1,1,0,0,0,1,0.704415,21,low
16,16,990097,2,Android,economy,no surge,1,1,1,1,0,1,5.766970,28,low


In [ ]:
# считаем среднее количество отмен по каждому пользователю
df_mean_cancelled = df_center_eve.loc[(df_center_eve['order_made'] == 1)].groupby(['user_id'])['user_cancelled'].mean().reset_index()
df_mean_cancelled.head()

,user_id,user_cancelled
0,100255,1.0
1,100313,0.0
2,100405,0.0
3,100434,0.0
4,100452,1.0


In [ ]:
#произведем рандомную разбивку пользователей на 2 группы
df_mean_cancelled['random_value'] = np.random.uniform(0,1, size = len(df_mean_cancelled['user_id']))


In [ ]:
def groups_gen(row):
  if row['random_value'] <=0.5:
    return(1)
  else: 
    return(2)

In [ ]:
df_mean_cancelled['group'] = df_mean_cancelled.apply(groups_gen, axis=1)

In [ ]:
df_mean_cancelled.head()

,user_id,user_cancelled,random_value,group
0,100255,1.0,0.702764,2
1,100313,0.0,0.397862,1
2,100405,0.0,0.876817,2
3,100434,0.0,0.266196,1
4,100452,1.0,0.629210,2


In [ ]:
df_mean_cancelled_g1 = df_mean_cancelled[df_mean_cancelled['group'] == 1]
df_mean_cancelled_g2 = df_mean_cancelled[df_mean_cancelled['group'] == 2]

In [ ]:
#проверим размеры выборок и выровняем их
len(df_mean_cancelled_g1['user_id']), len(df_mean_cancelled_g2['user_id'])

(5236, 5071)

In [ ]:
# первая группа имеет больше пользователей. Сократим их количество, чтобы оно было равно количеству пользователей во 2-й группе
group_volume = min(len(df_mean_cancelled_g1['user_id']), len(df_mean_cancelled_g2['user_id']))

In [ ]:
df_mean_cancelled_g1 = df_mean_cancelled_g1.iloc[0:group_volume-1]
df_mean_cancelled_g2 = df_mean_cancelled_g2.iloc[0:group_volume-1]

In [ ]:
len(df_mean_cancelled_g1['user_id']), len(df_mean_cancelled_g2['user_id'])

(5070, 5070)

Проверяем группы на однородность с помощью раногового критерия Манна-Уитни

In [ ]:
#Формируем H0 и H1:
#H0 Группы 1 и 2 являются однородными
#H1 Группы 1 и 2 имеют статистически значимые различия

In [ ]:
alpha = 0.05
stat, p = mannwhitneyu(df_mean_cancelled_g1['user_cancelled'], df_mean_cancelled_g2['user_cancelled'])
print('Statistics=%.3f, p=%.3f' % (stat, p))
if p > alpha:
  print('Группы 1 и 2 являются однородными (не отвергаем H0)')
else:
  print('Группы 1 и 2 имеют статистически значимые различия (отвергаем H0)')

Statistics=12820836.000, p=0.734
Группы 1 и 2 являются однородными (не отвергаем H0)
